In [1]:
import pandas as pd

In [2]:
df = pd.read_excel("Online Retail.xlsx")

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df.shape

(541909, 8)

In [5]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
df.dropna(inplace=True)

In [8]:
df.shape

(406829, 8)

In [10]:
df = df[df['Country'] == 'United Kingdom']
df.shape

(361878, 8)

In [11]:
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom


In [12]:
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

In [13]:
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00


In [16]:
# RFM Hesaplama
import datetime as dt
analysis_date = df['InvoiceDate'].max() + dt.timedelta(days=1)
analysis_date

Timestamp('2011-12-10 12:49:00')

In [24]:
rfm = df.groupby("CustomerID").agg({
    "InvoiceDate": lambda x: (analysis_date - x.max()).days,
    "InvoiceNo": "nunique",
    "TotalPrice": "sum"
})
#sum,mean,max,min
rfm.columns = ["Recency","Frequency","Monetary"]
#rfm

In [38]:
rfm["R_Score"] = pd.qcut(rfm["Recency"], 5, labels=[5,4,3,2,1], duplicates='drop')
rfm["F_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["M_Score"] = pd.qcut(rfm["Monetary"].rank(method="first"), 5, labels=[1,2,3,4,5])
rfm["RFM_Score"] = rfm["R_Score"].astype(str) + rfm["F_Score"].astype(str) + rfm["M_Score"].astype(str)
rfm

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score
CustomerID,,,,,,,
12346.0,326,2,0.00,1,2,1,121
12747.0,2,11,4196.01,5,5,5,555
12748.0,1,224,29072.10,5,5,5,555
12749.0,4,8,3868.20,5,5,5,555
12820.0,3,4,942.34,5,3,4,534
...,...,...,...,...,...,...,...
18280.0,278,1,180.60,1,2,1,121
18281.0,181,1,80.82,1,2,1,121
18282.0,8,3,176.60,5,3,1,531


In [39]:
from sklearn.cluster import KMeans



In [40]:
X = rfm[["Recency","Frequency","Monetary"]]
X

,Recency,Frequency,Monetary
CustomerID,,,
12346.0,326,2,0.00
12747.0,2,11,4196.01
12748.0,1,224,29072.10
12749.0,4,8,3868.20
12820.0,3,4,942.34
...,...,...,...
18280.0,278,1,180.60
18281.0,181,1,80.82
18282.0,8,3,176.60


In [41]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [42]:
kmeans = KMeans(n_clusters=5,random_state=42) # elbow method kullan!
rfm["KMeansCluster"] = kmeans.fit_predict(X_scaled)

In [43]:
rfm

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,KMeansCluster
CustomerID,,,,,,,,
12346.0,326,2,0.00,1,2,1,121,0
12747.0,2,11,4196.01,5,5,5,555,1
12748.0,1,224,29072.10,5,5,5,555,2
12749.0,4,8,3868.20,5,5,5,555,1
12820.0,3,4,942.34,5,3,4,534,1
...,...,...,...,...,...,...,...,...
18280.0,278,1,180.60,1,2,1,121,0
18281.0,181,1,80.82,1,2,1,121,0
18282.0,8,3,176.60,5,3,1,531,1


In [58]:
# Elbow method ile best n_cluster bulma
# Scatter plot oluşturma (kaç grup var? kaç eleamn var?)

rfm[rfm["KMeansCluster"] == 0]

,Recency,Frequency,Monetary,R_Score,F_Score,M_Score,RFM_Score,KMeansCluster,R_Score2
CustomerID,,,,,,,,,
12346.0,326,2,0.00,1,2,1,121,0,"(178.0, 374.0]"
12821.0,214,1,92.72,1,1,1,111,0,"(178.0, 374.0]"
12829.0,322,3,253.05,1,3,2,132,0,"(178.0, 374.0]"
12831.0,262,1,215.05,1,1,1,111,0,"(178.0, 374.0]"
12833.0,145,1,417.38,2,1,2,212,0,"(71.0, 178.0]"
...,...,...,...,...,...,...,...,...,...
18260.0,173,8,2595.00,2,5,5,255,0,"(71.0, 178.0]"
18262.0,140,1,149.48,2,2,1,221,0,"(71.0, 178.0]"
18269.0,358,2,138.90,1,3,1,131,0,"(178.0, 374.0]"


In [ ]:
# FAST API ile bu modeli api haline getirelim. Yeni bir müşterinin (RFM değeri hesaplanmış olsun.) RFM değerleri verildiğinde müşt